In [1]:
import pandas as pd
import string
from octis.preprocessing.preprocessing import Preprocessing
from octis.models.LDA import LDA

/opt/anaconda3/envs/patientx/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print("reading data...")

# read data
data_say_hello = pd.read_csv('/Users/vnarayan35/Documents/GitHub/PatientX.AI/existing_code/forum-crawler data/Say hello and introduce yourself.csv')
data_recently_diagnosed = pd.read_csv('/Users/vnarayan35/Documents/GitHub/PatientX.AI/existing_code/forum-crawler data/Recently diagnosed and early stages of dementia.csv')
data_memory_concerns = pd.read_csv("/Users/vnarayan35/Documents/GitHub/PatientX.AI/existing_code/forum-crawler data/Memory concerns and seeking a diagnosis.csv")
data_i_have_dementia = pd.read_csv("/Users/vnarayan35/Documents/GitHub/PatientX.AI/existing_code/forum-crawler data/I have dementia.csv")
data_i_have_partner = pd.read_csv("/Users/vnarayan35/Documents/GitHub/PatientX.AI/existing_code/forum-crawler data/I have a partner with dementia.csv")
data_i_care = pd.read_csv("/Users/vnarayan35/Documents/GitHub/PatientX.AI/existing_code/forum-crawler data/I care for a person with dementia.csv")


print("read data")

reading data...
read data


In [3]:
# combine data into single dataframe
dfs = [data_say_hello, data_recently_diagnosed, data_memory_concerns, data_i_have_dementia, data_i_have_partner, data_i_care]
forum_data_union = pd.concat(dfs, ignore_index=True)


SAMPLE_SIZE = 50

sample_data = forum_data_union.sample(SAMPLE_SIZE)

print("sampled")

sampled


In [5]:
# Save to TSV file

sample_data.to_csv(path_or_buf='/Users/vnarayan35/Documents/GitHub/PatientX.AI/existing_code/dataset/sample_data_final_fixed.tsv', index=False, sep='\t')

In [5]:
# group posts from the same forum/thread into one document and remove any line breaks
data_union_grouped = forum_data_union.groupby(['forum', 'thread_title'], as_index=False).agg({'post_message': ''.join})
data_union_grouped['post_message'] = data_union_grouped['post_message'].str.strip().replace(r'\n', ' ', regex=True)

In [8]:
# shape of grouped data
data_union_grouped.shape

(42020, 3)

In [7]:
# longest 'document' length
data_union_grouped['post_message'].str.len().max()

11678053

In [13]:
# brief look at what the aggregated data looks like
data_union_grouped.head(10)

,forum,thread_title,post_message
0,I care for a person with dementia,!Advice on dealing with delusions/regression -...,Hi everyone. This is my first post.\rI would l...
1,I care for a person with dementia,"""Continence"" problems",Just wondering if anyone's experienced this an...
2,I care for a person with dementia,"""Covering up"" mistakes",Like if she gets something wrong or says somet...
3,I care for a person with dementia,"""Do I like her"" Mum asked me about her new car...",Well day two was a disaster. \rCan you believe...
4,I care for a person with dementia,"""Dont ever discuss hallucinations,Distract""",This is what the consultant said last week. No...
5,I care for a person with dementia,"""Fight"" in My Mums's Nursing Home Today",While sitting with my mum today in the lounge ...
6,I care for a person with dementia,"""Getting someone into a care home""",That probably isn't a good title to a post but...
7,I care for a person with dementia,"""Going Home"" again.",Several years ago my father went through the p...
8,I care for a person with dementia,"""I don't want to be a burden""",My father periodically says he needs to buy ne...
9,I care for a person with dementia,"""I don't want to be here any more""",My father told his carer this evening that he ...


In [17]:
!pip install textblob

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 626.3/626.3 kB 18.8 MB/s eta 0:00:00


In [18]:
import pandas as pd
import spacy
from textblob import TextBlob  # or use autocorrect

# Load SpaCy model
nlp = spacy.load('en_core_web_sm')

# Function to remove non-English words and correct typos
def clean_message(message):
    # Process the message with SpaCy
    doc = nlp(message)
    
    # Remove non-English words (based on SpaCy's 'lang' attribute)
    filtered_tokens = [token.text for token in doc if token.lang_ == 'en']
    
    # Reconstruct the sentence from the filtered tokens
    filtered_message = ' '.join(filtered_tokens)
    
    # Fix typos using TextBlob (or autocorrect)
    corrected_message = str(TextBlob(filtered_message).correct())
    
    return corrected_message


In [29]:
# sample 1000 documents from the data - makes for a more manageable dataset to work with
trimmed = data_union_grouped.sample(1000)

In [30]:
# clean the sampled data by fixing typos and removing non-english words
trimmed['cleaned_message'] = trimmed['post_message'].apply(clean_message)

In [32]:
# save this cleaned data to a text file where documents are separated by a line break
trimmed['cleaned_message'].to_csv(r'./trimmed_cleaned.txt', header=None, index=None, sep='\n', mode='a')

In [21]:
data_union_grouped['post_message'] = data_union_grouped['post_message'].str.strip().replace(r'\r', ' ', regex=True) # minor cleaning
data_review_grouped = data_union_grouped['post_message'] # holds only the text from the dataset

In [23]:
# save posts to a txt file, documents separated by linebreaks
data_review_grouped.to_csv(r'./corpus_threads_combined.txt', header=None, index=None, sep='\n', mode='a')

In [6]:
# First pass at preprocessing the entire dataset

print("preprocessing...")

# preprocessing - remove whitespace, remove punctuation, convert to lowercase
preprocessor = Preprocessing(vocabulary=None, max_features=None,
                             remove_punctuation=True, punctuation=string.punctuation,
                             lemmatize=True, stopword_list='english',
                             min_chars=1, min_words_docs=0)

dataset = preprocessor.preprocess_dataset(documents_path=r'./corpus.txt')
print("done preprocessing")

preprocessing...


100%|██████████| 12118887/12118887 [21:23:34<00:00, 157.36it/s]    


created vocab
116455
done preprocessing


In [7]:
print("saving...")
dataset.save(path='./processed_dataset/')
print("done saving")

saving...
done saving


In [13]:
# Topic modeling with OCTIS LDA

model = LDA(num_topics=9)


print("training lda...")

# Train the model using default partitioning choice
output = model.train_model(dataset)

print("done training")

print(*list(output.keys()), sep="\n") # Print the output identifiers

for t in output['topics'][:5]:
  print(" ".join(t))

training lda...
done training
topic-word-matrix
topics
topic-document-matrix
test-topic-document-matrix
pay money house use work buy drive council cost key
hi uk www org alzheimers https attorney power play http
post hope sorry help find read support hello advice welcome
know think feel time thing mum like want try year
thank good x love send point wish hope xx hug


In [15]:
# print results again

for t in output['topics'][:20]:
  print(" ".join(t))

pay money house use work buy drive council cost key
hi uk www org alzheimers https attorney power play http
post hope sorry help find read support hello advice welcome
know think feel time thing mum like want try year
thank good x love send point wish hope xx hug
dementia year memory gp problem doctor mum medication ago diagnose
not s m good i t luck look write d
day go mum time get say night come week bed
care home need mum carer help social dad hospital service


In [17]:
from octis.evaluation_metrics.diversity_metrics import TopicDiversity
from octis.evaluation_metrics.coherence_metrics import Coherence
# Initialize metric
npmi = Coherence(texts=dataset.get_corpus(), topk=10, measure='c_npmi')
# Initialize metric
topic_diversity = TopicDiversity(topk=10)

In [18]:
# Retrieve metrics score
topic_diversity_score = topic_diversity.score(output)
print("Topic diversity: "+str(topic_diversity_score))

npmi_score = npmi.score(output)
print("Coherence: "+str(npmi_score))

Topic diversity: 0.9111111111111111
Coherence: 0.08240201214883959


In [33]:
# second preprocessor, changes in max doc freq and min doc freq
print("preprocessing...")

# preprocessing - remove whitespace, remove punctuation, convert to lowercase
preprocessor2 = Preprocessing(vocabulary=None, max_features=None,
                             remove_punctuation=True, punctuation=string.punctuation,
                             lemmatize=True, stopword_list='english',
                             min_chars=1, min_words_docs=0, num_processes=10, min_df=0.0001, max_df=0.7)

preprocessing...


In [5]:
# increase model max length
preprocessor2.spacy_model.max_length = 100000000

In [ ]:
dataset2 = preprocessor2.preprocess_dataset(documents_path=r'./corpus_threads_combined.txt')
print("done preprocessing")

In [ ]:
print("saving...")
dataset.save(path='./processed_dataset_2/')
print("done saving")

In [100]:
# remove custom stop words that aren't caught by spacy's model
from spacy.lang.en import stop_words

stop_words = list(stop_words.STOP_WORDS)
custom_stop_words = ['with', 'my', 'your', 'she', 'this', 'was', 'her', 'have', 'as', 'he', 'him', 'but', 'not', 'so', 'are', 'at', 'be', 'has', 'do', 'got', 'how', 'on', 'or', 'would', 'will', 'what', 'they', 'if', 'or', 'get', 'can', 'we', 'me', 'can', 'has', 'his', 'there', 'them', 'just', 'am', 'by', 'that', 'from', 'it', 'is', 'in', 'you', 'also', 'very', 'had', 'a', 'an', 'for']

stop_words += custom_stop_words

In [112]:
# third pass at preprocessing, changes in max df, min df, and using custom stop words
preprocessor3 = Preprocessing(vocabulary=None, max_features=None,
                             remove_punctuation=True, punctuation=string.punctuation,
                             lemmatize=True, stopword_list=custom_stop_words,
                             min_chars=1, min_words_docs=20, num_processes=10, min_df=0.01, max_df=0.5)

In [ ]:
preprocessor3.spacy_model.max_length = 100000000

In [35]:
# check max doc length amongst the sampled datapoints
trimmed['cleaned_message'].str.len().max()

98342

In [113]:
# preprocess the sampled data
dataset_trimmed_cleaned = preprocessor3.preprocess_dataset(documents_path=r'./trimmed_cleaned.txt')
print("done preprocessing")

100%|██████████| 9611/9611 [00:38<00:00, 251.28it/s]


created vocab
893
done preprocessing


In [99]:
# save dataset
print("saving...")
dataset_trimmed_cleaned.save(path='./processed_dataset_trimmed/')
print("done saving")

saving...
done saving


In [82]:
# NOTE THIS IS A CUSTOM LDA MODEL THAT BUILDS OFF THE OCTIS LDA - THIS ONE USES TF-IDF INSTEAD OF BAG OF WORDS

from octis.models.model import AbstractModel
import numpy as np
from gensim.models import ldamodel
import gensim.corpora as corpora
import octis.configuration.citations as citations
import octis.configuration.defaults as defaults
from gensim.models import TfidfModel


class LDA(AbstractModel):

    id2word = None
    id_corpus = None
    use_partitions = True
    update_with_test = False

    def __init__(
        self, num_topics=100, distributed=False, chunksize=2000,
        passes=1, update_every=1, alpha="symmetric", eta=None, decay=0.5,
        offset=1.0, eval_every=10, iterations=50, gamma_threshold=0.001,
            random_state=None):
        """
        Initialize LDA model

        Parameters
        ----------
        num_topics (int, optional) – The number of requested latent topics to
        be extracted from the training corpus.

        distributed (bool, optional) – Whether distributed computing should be
        used to accelerate training.

        chunksize (int, optional) – Number of documents to be used in each
        training chunk.

        passes (int, optional) – Number of passes through the corpus during
        training.

        update_every (int, optional) – Number of documents to be iterated
        through for each update. Set to 0 for batch learning, > 1 for
        online iterative learning.

        alpha ({numpy.ndarray, str}, optional) – Can be set to an 1D array of
        length equal to the number of expected topics that expresses our
        a-priori belief for the each topics’ probability. Alternatively
        default prior selecting strategies can be employed by supplying
        a string:

            ’asymmetric’: Uses a fixed normalized asymmetric prior of
            1.0 / topicno.

            ’auto’: Learns an asymmetric prior from the corpus
            (not available if distributed==True).

        eta ({float, np.array, str}, optional) – A-priori belief on word
        probability, this can be:

            scalar for a symmetric prior over topic/word probability,

            vector of length num_words to denote an asymmetric user defined
            probability for each word,

            matrix of shape (num_topics, num_words) to assign a probability
            for each word-topic combination,

            the string ‘auto’ to learn the asymmetric prior from the data.

        decay (float, optional) – A number between (0.5, 1] to weight what
        percentage of the previous lambda value is forgotten when each new
        document is examined.

        offset (float, optional) – Hyper-parameter that controls how much
        we will slow down the first steps the first few iterations.

        eval_every (int, optional) – Log perplexity is estimated every
        that many updates. Setting this to one slows down training by ~2x.

        iterations (int, optional) – Maximum number of iterations through the
        corpus when inferring the topic distribution of a corpus.

        gamma_threshold (float, optional) – Minimum change in the value of the
        gamma parameters to continue iterating.

        random_state ({np.random.RandomState, int}, optional) – Either a
        randomState object or a seed to generate one.s
        Useful for reproducibility.


        """
        super().__init__()
        self.hyperparameters = dict()
        self.hyperparameters["num_topics"] = num_topics
        self.hyperparameters["distributed"] = distributed
        self.hyperparameters["chunksize"] = chunksize
        self.hyperparameters["passes"] = passes
        self.hyperparameters["update_every"] = update_every
        self.hyperparameters["alpha"] = alpha
        self.hyperparameters["eta"] = eta
        self.hyperparameters["decay"] = decay
        self.hyperparameters["offset"] = offset
        self.hyperparameters["eval_every"] = eval_every
        self.hyperparameters["iterations"] = iterations
        self.hyperparameters["gamma_threshold"] = gamma_threshold
        self.hyperparameters["random_state"] = random_state

    def info(self):
        """
        Returns model informations
        """
        return {
            "citation": citations.models_LDA,
            "name": "LDA, Latent Dirichlet Allocation"
        }

    def hyperparameters_info(self):
        """
        Returns hyperparameters informations
        """
        return defaults.LDA_hyperparameters_info

    def set_hyperparameters(self, **kwargs):
        """
        Set model hyperparameters
        """
        super().set_hyperparameters(**kwargs)
        # Allow alpha to be a float in case of symmetric alpha
        if "alpha" in kwargs:
            if isinstance(kwargs["alpha"], float):
                self.hyperparameters["alpha"] = [
                    kwargs["alpha"]
                ] * self.hyperparameters["num_topics"]

    def partitioning(self, use_partitions, update_with_test=False):
        """
        Handle the partitioning system to use and reset the model to perform
        new evaluations

        Parameters
        ----------
        use_partitions: True if train/set partitioning is needed, False
                        otherwise
        update_with_test: True if the model should be updated with the test set,
                          False otherwise
        """
        self.use_partitions = use_partitions
        self.update_with_test = update_with_test
        self.id2word = None
        self.id_corpus = None

    def train_model(self, dataset, hyperparams=None, top_words=10):
        """
        Train the model and return output

        Parameters
        ----------
        dataset : dataset to use to build the model
        hyperparams : hyperparameters to build the model
        top_words : if greater than 0 returns the most significant words for
                    each topic in the output (Default True)
        Returns
        -------
        result : dictionary with up to 3 entries,
                 'topics', 'topic-word-matrix' and
                 'topic-document-matrix'
        """
        if hyperparams is None:
            hyperparams = {}

        if self.use_partitions:
            train_corpus, test_corpus = dataset.get_partitioned_corpus(
                use_validation=False)
        else:
            train_corpus = dataset.get_corpus()

        if self.id2word is None:
            self.id2word = corpora.Dictionary(dataset.get_corpus())

        if self.id_corpus is None:
            self.id_corpus = [self.id2word.doc2bow(document)
                              for document in train_corpus]

        if "num_topics" not in hyperparams:
            hyperparams["num_topics"] = self.hyperparameters["num_topics"]

        # Allow alpha to be a float in case of symmetric alpha
        if "alpha" in hyperparams:
            if isinstance(hyperparams["alpha"], float):
                hyperparams["alpha"] = [
                    hyperparams["alpha"]
                ] * hyperparams["num_topics"]

        #### changes #####
        print("using tf-idf")
        tfidf_model = TfidfModel(self.id_corpus)
        tfidf_corpus = tfidf_model[self.id_corpus]
        hyperparams["corpus"] = tfidf_corpus
        
        
        # hyperparams["corpus"] = self.id_corpus
        
        hyperparams["id2word"] = self.id2word
        self.hyperparameters.update(hyperparams)

        self.trained_model = ldamodel.LdaModel(**self.hyperparameters)

        result = {}

        result["topic-word-matrix"] = self.trained_model.get_topics()

        if top_words > 0:
            topics_output = []
            for topic in result["topic-word-matrix"]:
                top_k = np.argsort(topic)[-top_words:]
                top_k_words = list(reversed([self.id2word[i] for i in top_k]))
                topics_output.append(top_k_words)
            result["topics"] = topics_output

        result["topic-document-matrix"] = self._get_topic_document_matrix()

        if self.use_partitions:
            new_corpus = [self.id2word.doc2bow(
                document) for document in test_corpus]
            if self.update_with_test:
                self.trained_model.update(new_corpus)
                self.id_corpus.extend(new_corpus)

                result["test-topic-word-matrix"] = (
                    self.trained_model.get_topics())

                if top_words > 0:
                    topics_output = []
                    for topic in result["test-topic-word-matrix"]:
                        top_k = np.argsort(topic)[-top_words:]
                        top_k_words = list(
                            reversed([self.id2word[i] for i in top_k]))
                        topics_output.append(top_k_words)
                    result["test-topics"] = topics_output

                result["test-topic-document-matrix"] = (
                    self._get_topic_document_matrix())

            else:
                test_document_topic_matrix = []
                for document in new_corpus:
                    document_topics_tuples = self.trained_model[document]
                    document_topics = np.zeros(
                        self.hyperparameters["num_topics"])
                    for single_tuple in document_topics_tuples:
                        document_topics[single_tuple[0]] = single_tuple[1]

                    test_document_topic_matrix.append(document_topics)
                result["test-topic-document-matrix"] = np.array(
                    test_document_topic_matrix).transpose()
        return result

    def _get_topics_words(self, topk):
        """
        Return the most significative words for each topic.
        """
        topic_terms = []
        for i in range(self.hyperparameters["num_topics"]):
            topic_words_list = []
            for word_tuple in self.trained_model.get_topic_terms(i, topk):
                topic_words_list.append(self.id2word[word_tuple[0]])
            topic_terms.append(topic_words_list)
        return topic_terms

    def _get_topic_document_matrix(self):
        """
        Return the topic representation of the
        corpus
        """
        doc_topic_tuples = []
        for document in self.id_corpus:
            doc_topic_tuples.append(
                self.trained_model.get_document_topics(document,
                                                       minimum_probability=0))

        topic_document = np.zeros((
            self.hyperparameters["num_topics"],
            len(doc_topic_tuples)))

        for ndoc in range(len(doc_topic_tuples)):
            document = doc_topic_tuples[ndoc]
            for topic_tuple in document:
                topic_document[topic_tuple[0]][ndoc] = topic_tuple[1]
        return topic_document

In [134]:
# train custom model on sampled data

model_trim = LDA(num_topics=6)


print("training lda...")

# Train the model using default partitioning choice
output = model_trim.train_model(dataset_trimmed_cleaned, top_words=50)

print("done training")

print(*list(output.keys()), sep="\n") # Print the output identifiers

for t in output['topics'][:20]:
  print(" ".join(t))

training lda...
using tf-idf
done training
topic-word-matrix
topics
topic-document-matrix
test-topic-document-matrix
care nursing home memory people sum dementia eat cares go some time then one being into meditation say funding help want when no other which staff their option could out homes ca mother who difficult way needs think etc been give any work now day term stages someone need something
care home think sum all no could time been up who about homes many family feel years one help did only some day house dementia does our now any room like someone really call after needs which own mother know more when people live other dad move right were may
sum dementia home all care dad time could about know things when help been now up too husband go did well here day may were then see some out one feel make more does think people should said best really much memory our who mother want years no still like
did all who home now care no sum said when go us because only up know over out been on

In [108]:
# print results
for t in output['topics']:
  print(" ".join(t))
  print("\n")

needs home may point sum pain out were care which comes about no think been mother need go more sorry sums any say said know keep off too much accept


sum phone help 1 some after does think always advice down anyone go did night may dad home like care small taking no up room time given now give able


night any bed husband dementia years which dad vascular been here why time could back does out day who put hospital support no memory really after now diagnosed go around


care home about ca no then sister time said did one when our their help keep down years been much week look into things all see dad because carer though




In [109]:
# TWO FUNCTIONS FOR POST-PROCESSING

from typing import List

# take the outputted lists and return only words that are unique to their respective lists
def unique_words(lists: List[List[str]]) -> List[List[str]]:
    # Convert each sublist to a set for easier manipulation
    sets = [set(sublist) for sublist in lists]
    unique_lists = []

    for i, word_set in enumerate(sets):
        # Calculate the union of all sets except the current one
        other_words = set().union(*[s for j, s in enumerate(sets) if j != i])
        # Find words unique to the current set
        unique_lists.append(list(word_set - other_words))

    return unique_lists


# take outputted words and keep only words that show up in less than <threshold> percent of topics
def unique_words_threshold(lists: List[List[str]], threshold: float) -> List[List[str]]:
     # Total number of lists
    num_lists = len(lists)
    # Calculate the threshold count based on the percentage
    max_allowed_count = int(threshold * num_lists)
     
     
    # Convert each sublist to a set for easier manipulation
    sets = [set(sublist) for sublist in lists]
    unique_lists = []
    
    for i, word_set in enumerate(sets):
        new_set = []
        for word in word_set:
            other_counts = 0
            for j, other_set in enumerate(sets):
                if word in other_set and i != j:
                    other_counts += 1
        
            if other_counts <= max_allowed_count:
                new_set.append(word)
        unique_lists.append(new_set)

    return unique_lists

In [115]:
# print unique words
for l in unique_words(output['topics']):
    print(" ".join(l))
    print("\n")

need cares needs own wife see


someone call weeks months hospital last 10 tea again phone pain taking should told agency after


times really being memory than


where house never off thought down car 3 made shone first even week using family seems ca




In [149]:
# print 20 percent threshold results
unique_threshold_20 = unique_words_threshold(output['topics'], 0.2)

for l in unique_threshold_20:
    print(" ".join(l))
    print("\n")

their homes etc eat being option way staff say stages someone need want memory ca something cares needs term meditation work give other funding nursing difficult


homes many call right someone room house move live family needs other own only


best well too want memory husband should here make


happening because over tea last mind going using night thing life husband us own only good


way forms need nothing going happy take should love back days person wife good year


5 week bed seems house too because over take food bad getting used morning back hospital days pain put today than


